In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [ ]:
import tensorboard
tensorboard.__version__

## Conceptual graph

In addition to the execution graph, TensorBoard also displays a **conceptual graph**. This is a view of just the Keras model. This may be useful if you’re reusing a saved model and you want to examine or validate its structure.

To see the conceptual graph, select the “keras” tag. For this example, you’ll see a collapsed **Sequential** node. Double-click the node to see the model’s structure:

## Graphs of tf.functions

The examples so far have described graphs of Keras models, where the graphs have been created by defining Keras layers and calling Model.fit().

You may encounter a situation where you need to use the `tf.function` annotation to ["autograph"](https://www.tensorflow.org/guide/function), i.e., transform, a Python computation function into a high-performance TensorFlow graph. For these situations, you use **TensorFlow Summary Trace API** to log autographed functions for visualization in TensorBoard.

To use the Summary Trace API:

*   Define and annotate a function with `tf.function`
*   Use `tf.summary.trace_on()` immediately before your function call site.
*    Add profile information (memory, CPU time) to graph by passing `profiler=True`
*   With a Summary file writer, call `tf.summary.trace_export()` to save the log data

You can then use TensorBoard to see how your function behaves.


In [ ]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

In [ ]:
%tensorboard --logdir logs/func

# My Own Test Case Monthly Save

In [ ]:
# The function to be traced.
@tf.function
def my_func(My_income, My_parter_income, Total_cost):
  # A simple hand-rolled layer.
  return tf.add(My_income, My_parter_income) - Total_cost

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/myfunc/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
My_income = tf.random.uniform((3, 3))
My_parter_income = tf.random.uniform((3, 3))
Total_cost = 100

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(My_income, My_parter_income, Total_cost)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/myfunc2 

In [ ]:
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/myfunc2/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

C_1 = tf.constant(5.0)
C_2 = tf.constant(1.0)
C_3 = tf.constant(2.0)

@tf.function
def get_ratio(C1,C2,C3):
    golden_ratio = (tf.sqrt(C1) + C2)/C3
    return golden_ratio

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
golden_ratio = get_ratio(C_1,C_2,C_3)
# Call only one tf.function when tracing.
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace2",
      step=0,
      profiler_outdir=logdir)